In [8]:
import os
import pandas as pd
import natsort
data_dir =  '../cropped/cropped'
data_per = os.listdir(data_dir)

In [14]:
def max_frame(data_per):
    max_frames = 0
    for person in data_per:
        if person == '.DS_Store':
            continue
        words_dir = f'{data_dir}/{person}/words'
        Id = os.listdir(words_dir)
        for id in Id :
            id_dir = f'{data_dir}/{person}/words/{id}'
            num = os.listdir(id_dir)
            for n  in num:
                #print("n is", n)
                words_dir_2 = f'{data_dir}/{person}/words/{id}/{n}'
                frames = os.listdir(words_dir_2)
                #print(len(frames))
                max_frames = max(len(frames), max_frames)
    return max_frames

In [41]:
def concat_x_data(x_data, x_data_):
    if x_data_.shape[0] == max_frame(data_per):
        return np.concatenate([x_data, x_data_], axis = 1) 
    else: 
        pad = max_frame(data_per) - x_data_.shape[1]
        print("pad for ",pad)
        zero_pad = np.zeros((1, pad, 12288))
        print(zero_pad.shape)
        print(x_data_.shape)
        data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
        return data

In [44]:
from tensorflow.keras.preprocessing import image
import numpy as np

for person in natsort.natsorted(data_per):
    if person == '.DS_Store':
        continue
    #phrases_dir = f'{data_dir}/{person}/phrases'
    words_dir = f'{data_dir}/{person}/words'
    Id = os.listdir(words_dir)
    for sentence in natsort.natsorted(Id) :
        id_dir = f'{data_dir}/{person}/words/{sentence}'
        num = os.listdir(id_dir)
        for n  in natsort.natsorted(num):
            print("person: {}, sentence: {}, n: {} ".format(person, sentence, n))
            words_dir_2 = f'{data_dir}/{person}/words/{sentence}/{n}'
            frames = os.listdir(words_dir_2)
            x_data_ = np.empty(3)
            x_data_aug = np.empty(3)
            for frame in natsort.natsorted(frames):
                img_path = f'{words_dir_2}/{frame}'
                
                # 이미지 하나 당 flatten 해주어서 12288 차원으로 만들어 줌
                img = image.load_img(img_path, target_size=(64, 64))
                img_tensor = image.img_to_array(img) #tensor로 type 변경
                img_tensor = img_tensor.flatten() # flatten: 12288 차원
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 12288) 차원
                
                # frame 별 묶기
                if frame == 'color_001.jpg':
                    x_data_ = img_tensor
                    x_data_aug = DataAugmentation().generate(img)
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis = 0)
                    img_augmetned = DataAugmentation.generate(img)
                    x_data_aug = np.concatenate([x_data_aug, img_augmetned], axis=0)
                
                print(x_data_.shape)
            '''
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)
            if n == '01':
                x_data = x_data_
            else:
                x_data = np.concatenate([x_data, x_data_], axis = 0)
            '''
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)  #[1, frame, image_size]
            x_data_aug = np.expand_dims(x_data_aug, axis=0)
            if (n == '01' and sentence == '01' and person == 'F01'):
                x_data = concat_x_data(x_data, x_data_)
                x_data = concat_x_data(x_data, x_data_aug)
            else:
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = concat_x_data(x_data, x_data_)
                    x_data = concat_x_data(x_data, x_data_aug)
                else: 
                    data = concat_x_data(x_data, x_data_)
                    x_data = np.concatenate([x_data, data], axis = 0)
                    data_aug = concat_x_data(x_data, x_data_aug)
                    x_data = np.concatenate([x_data, data_aug], axis = 0)
            print(x_data.shape)

person: F01, sentence: 01, n: 01 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
pad for  10
(1, 10, 12288)
(1, 10, 12288)
pad for  10
(1, 10, 12288)
(1, 10, 12288)
(1, 20, 12288)
person: F01, sentence: 01, n: 02 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
(3, 20, 12288)
person: F01, sentence: 01, n: 03 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
pad for  10
(1, 10, 12288)
(1, 10, 12288)
pad for  10
(1, 10, 12288)
(1, 10, 12288)
(5, 20, 12288)
person: F01, sentence: 01, n: 04 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
pad for  11
(1, 11, 12288)
(1, 9, 12288)
pad for  11
(1, 11, 12288)
(1, 9, 12288)
(7, 20, 12288)
person: F01, sentence: 01, n: 05 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 122

(75, 20, 12288)
person: F02, sentence: 01, n: 09 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
(77, 20, 12288)
person: F02, sentence: 01, n: 10 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
(79, 20, 12288)
person: F02, sentence: 02, n: 01 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
pad for  13
(1, 13, 12288)
(1, 7, 12288)
(81, 20, 12288)
person: F02, sentence: 02, n: 02 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
pad for  11
(1, 11, 12288)
(1, 9, 12288)
pad for  11
(1, 11, 12288)
(1, 9, 12288)
(83, 20, 12288)
person: F02, sentence: 02, n: 03 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
pad 

pad for  3
(1, 3, 12288)
(1, 17, 12288)
(139, 20, 12288)
person: F04, sentence: 02, n: 01 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
(14, 12288)
(15, 12288)
(16, 12288)
(17, 12288)
(18, 12288)
(19, 12288)
pad for  1
(1, 1, 12288)
(1, 19, 12288)
pad for  1
(1, 1, 12288)
(1, 19, 12288)
(141, 20, 12288)
person: F04, sentence: 02, n: 02 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
(14, 12288)
(15, 12288)
(16, 12288)
pad for  4
(1, 4, 12288)
(1, 16, 12288)
pad for  4
(1, 4, 12288)
(1, 16, 12288)
(143, 20, 12288)
person: F04, sentence: 02, n: 03 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
(14, 12288)
(15, 12288)
(16, 12288)
(17, 12288)
pad for  3
(1, 3, 12288)
(1, 17, 12288)
pad for  3

(193, 20, 12288)
person: F05, sentence: 01, n: 08 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
pad for  9
(1, 9, 12288)
(1, 11, 12288)
pad for  9
(1, 9, 12288)
(1, 11, 12288)
(195, 20, 12288)
person: F05, sentence: 01, n: 09 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
pad for  9
(1, 9, 12288)
(1, 11, 12288)
pad for  9
(1, 9, 12288)
(1, 11, 12288)
(197, 20, 12288)
person: F05, sentence: 01, n: 10 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
pad for  7
(1, 7, 12288)
(1, 13, 12288)
pad for  7
(1, 7, 12288)
(1, 13, 12288)
(199, 20, 12288)
person: F05, sentence: 02, n: 01 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
pad for  7
(1, 7

pad for  4
(1, 4, 12288)
(1, 16, 12288)
(253, 20, 12288)
person: F06, sentence: 01, n: 08 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
(14, 12288)
(15, 12288)
pad for  5
(1, 5, 12288)
(1, 15, 12288)
pad for  5
(1, 5, 12288)
(1, 15, 12288)
(255, 20, 12288)
person: F06, sentence: 01, n: 09 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
(14, 12288)
pad for  6
(1, 6, 12288)
(1, 14, 12288)
pad for  6
(1, 6, 12288)
(1, 14, 12288)
(257, 20, 12288)
person: F06, sentence: 01, n: 10 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
(12, 12288)
(13, 12288)
(14, 12288)
pad for  6
(1, 6, 12288)
(1, 14, 12288)
pad for  6
(1, 6, 12288)
(1, 14, 12288)
(259, 20, 12288)
person: F06, sentence: 02, n: 01 
(1, 12288)
(2, 12288)
(3, 1

In [37]:
# ref: https://m.blog.naver.com/PostView.nhn?blogId=isu112600&logNo=221582003889&proxyReferer=https:%2F%2Fwww.google.com%2F
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataAugmentation:

    @classmethod
    def generate(self, img):
        img_array = image.array_to_img(img)
        samples = np.expand_dims(img_array, axis=0)
        
        # generatior 생성
        datagen = ImageDataGenerator(
            width_shift_range=[-10,10], 
            height_shift_range=0.5, 
            rotation_range=45)
        
        it = datagen.flow(samples, batch_size=1)
        augmented = it.next()
        img_augmented = augmented[0].flatten()
        img_augmented /= 255. 
        img_augmented = np.expand_dims(img_augmented, 0)

        return img_augmented

#     @classmethod
#     def generate(self, img, count):
#         # 원본 img 정규화
#         img_tensor = img.flatten()
#         img_tensor /= 255.
#         img_tensor = np.expand_dims(img_tensor, axis=0)
#         self.augmentedData = img_tenor
        
#         # augmentation
#         self.data = np.expand_dims(img, axis=0)
#         self.width_shift_range(self, count)
#         self.height_shift_range(self, count)
#         self.random_rotation(self, count)
        
#         return self.augmentedData
    
    
#     # 좌우로 shift
#     def width_shift_range(self, count):
#         # generatior 생성
#         datagen = ImageDataGenerator(width_shift_range=[-20,20])
        
#         # iterator 준비
#         it = datagen.flow(self.data, batch_size=1)
        
#         for i in range(count):
#             x_batch = it.next()
#             for x in x_batch:
#                 x_flatten = x.flatten()
#                 x_flatten /= 255.
#                 x_flatten = np.expand_dims(x_flatten, 0)
                
#                 self.augmentedData = np.concatenate([self.augmentedData, x_flatten], axis=0)
                    
#     # 상하로 shift
#     def height_shift_range(self, count):
#         # generatior 생성
#         datagen = ImageDataGenerator(height_shift_range=0.5)
        
#         # iterator 준비
#         it = datagen.flow(self.data, batch_size=1)
        
#         for i in range(count):
#             x_batch = it.next()
#             for x in x_batch:
#                 x_flatten = x.flatten()
#                 x_flatten /= 255.
#                 x_flatten = np.expand_dims(x_flatten, 0)
                
#                 self.augmentedData = np.concatenate([self.augmentedData, x_flatten], axis=0)    
                
#     # 랜덤 각도 돌리기
#     def random_rotation(self, count):
#         # generatior 생성
#         datagen = ImageDataGenerator(rotation_range=90)
        
#         # iterator 준비
#         it = datagen.flow(self.data, batch_size=1)
        
#         for i in range(count):
#             x_batch = it.next()
#             for x in x_batch:
#                 x_flatten = x.flatten()
#                 x_flatten /= 255.
#                 x_flatten = np.expand_dims(x_flatten, 0)
                
#                 self.augmentedData = np.concatenate([self.augmentedData, x_flatten], axis=0)